In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "2g")
    .config("spark.executor.memory", "4g")
    .getOrCreate()
)

22/09/14 10:32:06 WARN Utils: Your hostname, Macintosh.local resolves to a loopback address: 127.0.0.1; using 10.12.247.25 instead (on interface en0)
22/09/14 10:32:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/14 10:32:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Read the given datasets

In [2]:
ori_transaction1 = spark.read.parquet('../data/tables/transactions_20210228_20210827_snapshot').sort('order_datetime')
ori_transaction2 = spark.read.parquet('../data/tables/transactions_20210828_20220227_snapshot').sort('order_datetime')
ori_transaction3 = spark.read.parquet('../data/tables/transactions_20220228_20220828_snapshot').sort('order_datetime')

#read the parquet ori_transaction
ori_transaction = ori_transaction1.union(ori_transaction2)
ori_transaction = ori_transaction.union(ori_transaction3)
ori_transaction.groupby('merchant_abn').count().count()

4422

In [3]:
tbl_merchants = spark.read.parquet('../data/tables/tbl_merchants.parquet')
tbl_merchants.count()

4026

In [4]:
consumer_detail = spark.read.parquet('../data/tables/consumer_user_details.parquet')

In [5]:
import pandas as pd
tbl_consumer = spark.read.option("delimiter", "|").option("header",True).csv("../data/tables/tbl_consumer.csv")

In [6]:
import pandas as pd
tbl_consumer = spark.read.option("delimiter", "|").option("header",True).csv("../data/tables/tbl_consumer.csv")

# Merchant datasets

In [7]:
from pyspark.sql.functions import round
grouped_transaction = ori_transaction.groupBy('merchant_abn').agg(F.sum('dollar_value').alias('Amount'), F.count('dollar_value').alias('Count')).sort('merchant_abn')
grouped_transaction.drop(F.col('order_id'))
grouped_transaction = grouped_transaction.withColumn('Monthly Amount', round(grouped_transaction['Amount']/12, 2))
#grouped_transaction = grouped_transaction.withColumn('Round Monthly Amount', round(grouped_transaction['Monthly Amount'], 2))
grouped_transaction = grouped_transaction.withColumn('Monthly Count', round(grouped_transaction['Count']/12, 2))
grouped_transaction

22/09/14 10:32:17 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:32:17 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:32:17 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:32:17 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:32:17 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:32:17 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:32:17 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:32:17 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/09/14 10:32:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:32:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:32:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:32:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:32:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:32:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:32:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:32:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


merchant_abn,Amount,Count,Monthly Amount,Monthly Count
10023283211,703277.7114509274,3261,58606.48,271.75
10142254217,118356.1460726035,3036,9863.01,253.0
10165489824,56180.473857030534,5,4681.71,0.42
10187291046,39693.730387434036,336,3307.81,28.0
10192359162,177980.50545638183,385,14831.71,32.08
10206519221,358222.999801654,9621,29851.92,801.75
10255988167,322474.116585636,832,26872.84,69.33
10264435225,587225.9354689713,5026,48935.49,418.83
10279061213,166058.19172755213,555,13838.18,46.25
10323485998,1327030.2646789101,10335,110585.86,861.25


In [11]:
grouped_transaction_pd = grouped_transaction.toPandas()
tbl_merchants_pd = tbl_merchants.toPandas()
merchant = pd.merge(grouped_transaction_pd, tbl_merchants_pd)
merchant.to_parquet("../data/curated/merchant.parquet")

In [12]:
for i in range(int(merchant['tags'].count())):
    merchant['tags'].iloc[i] = merchant['tags'].iloc[i].replace(r'[', r'(').replace(r']', r')')


/var/folders/63/097df8mn6r1d0btnhwhjpkr40000gn/T/ipykernel_4878/3340515154.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merchant['tags'].iloc[i] = merchant['tags'].iloc[i].replace(r'[', r'(').replace(r']', r')')


In [13]:
#merchant['tags'].str.split(pat, n=-1, expand=False)
# to split into multiple columns by delimiter
merchant_tags = merchant['tags'].str.split(')', expand=True)

In [14]:
for row in range(int(len(merchant_tags))):
    for col in range(3):
        merchant_tags.iloc[row,col] = merchant_tags.iloc[row,col].replace(r'((', r'').replace(r', (', r'').replace(r'take rate:', r'')
merchant_tags.rename(columns = {0 : 'Store type', 1 : 'Revenue levels', 2 : 'Take rate'}, inplace = True)
merchant_tags = merchant_tags[['Store type', 'Revenue levels', 'Take rate']]

In [15]:
import pyspark.sql.functions as f
merchant[['Store type', 'Revenue levels', 'Take rate']] = merchant_tags[['Store type', 'Revenue levels', 'Take rate']]
#merchant.drop(columns=['tags'])
merchant['Store type'] = merchant['Store type'].str.lower()

for i in range(len(merchant)):
    merchant['Store type'][i] = ' '.join(merchant['Store type'][i].split())

#merchant
merchant.groupby('Store type').count()

/var/folders/63/097df8mn6r1d0btnhwhjpkr40000gn/T/ipykernel_4878/697679775.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merchant['Store type'][i] = ' '.join(merchant['Store type'][i].split())


,merchant_abn,Amount,Count,Monthly Amount,Monthly Count,name,tags,Revenue levels,Take rate
Store type,,,,,,,,,
"antique shops - sales, repairs, and restoration services",129,129,129,129,129,129,129,129,129
art dealers and galleries,112,112,112,112,112,112,112,112,112
artist supply and craft shops,193,193,193,193,193,193,193,193,193
bicycle shops - sales and service,170,170,170,170,170,170,170,170,170
"books, periodicals, and newspapers",164,164,164,164,164,164,164,164,164
"cable, satellite, and other pay television and radio services",175,175,175,175,175,175,175,175,175
"computer programming , data processing, and integrated systems design services",191,191,191,191,191,191,191,191,191
"computers, computer peripheral equipment, and software",181,181,181,181,181,181,181,181,181
"digital goods: books, movies, music",195,195,195,195,195,195,195,195,195


# Consumer datasets

In [17]:
consumer_detail_pd = consumer_detail.toPandas()
tbl_consumer_pd = tbl_consumer.toPandas()
tbl_consumer_pd['consumer_id']=tbl_consumer_pd['consumer_id'].astype(int)
consumer = pd.merge(tbl_consumer_pd, consumer_detail_pd)
consumer

,name,address,state,postcode,gender,consumer_id,user_id
0,Yolanda Williams,413 Haney Gardens Apt. 742,WA,6935,Female,1195503,1
1,Mary Smith,3764 Amber Oval,NSW,2782,Female,179208,2
2,Jill Jones MD,40693 Henry Greens,NT,862,Female,1194530,3
3,Lindsay Jimenez,00653 Davenport Crossroad,NSW,2780,Female,154128,4
4,Rebecca Blanchard,9271 Michael Manors Suite 651,WA,6355,Female,712975,5
...,...,...,...,...,...,...,...
499994,Jessica Avila,508 Miranda Overpass Apt. 218,QLD,4400,Female,1385608,499995
499995,Steven Thornton,7913 Schwartz Mission Suite 483,VIC,3097,Undisclosed,1466964,499996
499996,Christy Smith,5681 Zachary Mountain Apt. 060,NSW,2756,Undisclosed,1253484,499997
499997,Donna Sutton,54140 Jacob Point,VIC,3989,Female,175005,499998


In [18]:
grouped_transaction = ori_transaction.groupBy('user_id').agg(F.avg('dollar_value').alias('avg'), F.sum('dollar_value').alias('sum'), F.count('dollar_value').alias('count')).sort('user_id')
grouped_transaction.drop(F.col('order_id'))
grouped_transaction_df = grouped_transaction.toPandas()
consumer_transaction = pd.merge(consumer, grouped_transaction_df)
consumer_transaction.to_parquet("../data/curated/consumer_transaction.parquet")

22/09/14 10:34:31 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:34:31 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:34:31 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:34:31 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:34:31 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:34:31 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:34:31 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:34:31 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


## Guess postcode for the merchant

In [19]:
merchant_abn_and_consumer_id = ori_transaction['merchant_abn', 'user_id']
#merchant_and_consumer_postcode['consumer postcode'] = c
merchant_abn_and_consumer_id
user_id_and_postcode = consumer[['postcode','user_id']]

user_id_and_postcode=spark.createDataFrame(user_id_and_postcode) 
#merchant_and_consumer_postcode = spark.merge(merchant_abn_and_consumer_id, user_id_and_postcode)
merchant_and_consumer_postcode = merchant_abn_and_consumer_id.join(user_id_and_postcode,['user_id'])
merchant_and_consumer_postcode = merchant_and_consumer_postcode['merchant_abn', 'postcode']
#merchant_and_consumer_postcode

#https://stackoverflow.com/questions/36654162/mode-of-grouped-data-in-pyspark
counts = merchant_and_consumer_postcode.groupBy(['merchant_abn', 'postcode']).count().alias('counts')
merchant_postcode = (counts
          .groupBy('merchant_abn')
          .agg(F.max(F.struct(F.col('count'),
                              F.col('postcode'))).alias('max'))
          .select(F.col('merchant_abn'), F.col('max.postcode'))
         )
merchant_postcode


22/09/14 10:34:42 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:34:42 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:34:42 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:34:42 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:34:42 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:34:42 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:34:42 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:34:42 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/09/14 10:34:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:34:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:34:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:34:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:34:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:34:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:34:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:34:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


merchant_abn,postcode
10023283211,5582
10142254217,6438
10165489824,6421
10187291046,7054
10192359162,9015
10206519221,5273
10255988167,3287
10264435225,6001
10279061213,6321
10323485998,4856


## Percentage of regular customer

In [20]:
ori_transaction_1 = ori_transaction.groupby('merchant_abn', 'user_id').count()
o_t = ori_transaction.groupby('merchant_abn').agg(F.count('user_id').alias('cnt'))
ori_con = ori_transaction_1.join(o_t, ori_transaction_1.merchant_abn == o_t.merchant_abn).drop(o_t.merchant_abn)
ori_con_drop = ori_con.filter(F.col("count") > 10)
ori_con_fix = ori_con_drop.groupby('merchant_abn').agg(F.count('user_id').alias('fix_cus_num'), F.avg('cnt').alias('total_cus_num'))
ori_con_fix_prob = ori_con_fix.withColumn("fix_cus_prob", ori_con_fix.fix_cus_num /ori_con_fix.total_cus_num)
ori_con_fix_prob

22/09/14 10:34:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:34:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:34:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:34:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:34:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:34:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:34:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:34:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/09/14 10:34:55 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:34:55 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:34:55 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:34:55 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:34:55 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:34:55 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:34:55 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:34:55 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:34:55 WARN RowBasedKeyValueBatch: Calling spill() on

22/09/14 10:35:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:35:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:35:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:35:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:35:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:35:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:35:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:35:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/09/14 10:35:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:35:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:35:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:35:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:35:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:35:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:35:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:35:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 10:35:04 WARN RowBasedKeyValueBatch: Calling spill() on

22/09/14 10:35:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


merchant_abn,fix_cus_num,total_cus_num,fix_cus_prob
49891706470,10913,247526.0,0.044088297794979116
89726005175,6971,215963.0,0.032278677366030295
32234779638,246,116773.0,0.00210665136632612
43186523025,5280,200913.0,0.026280031655492676
49505931725,2,68240.0,2.930832356389214...
60956456424,41,90705.0,4.520147731657571...
45629217853,8441,228255.0,0.0369805699765613
86578477987,13959,272674.0,0.05119299969927459
63290521567,3411,181876.0,0.018754536057533706
80324045558,4865,196840.0,0.024715504978662872
